# Tokenize
テキストをトークン化して、Transformerで処理可能な形式にする。

## 文字トークン化
最も単純なトークン化手法の一つで、各文字をモデルに与える。  
スペルミスや珍しい単語の処理に役立つ手法だが、デメリットとしてテキスト構造を無視してしまうことと、膨大な計算量、メモリ、データが必要になる等が挙げられる。

In [37]:
text = "Tokenizing text is a core task of NLP."
tokenized_text = list(text)
tokenized_text

['T',
 'o',
 'k',
 'e',
 'n',
 'i',
 'z',
 'i',
 'n',
 'g',
 ' ',
 't',
 'e',
 'x',
 't',
 ' ',
 'i',
 's',
 ' ',
 'a',
 ' ',
 'c',
 'o',
 'r',
 'e',
 ' ',
 't',
 'a',
 's',
 'k',
 ' ',
 'o',
 'f',
 ' ',
 'N',
 'L',
 'P',
 '.']

In [38]:
token2idx = {ch: idx for idx, ch in enumerate(sorted(set(tokenized_text)))}
token2idx

{' ': 0,
 '.': 1,
 'L': 2,
 'N': 3,
 'P': 4,
 'T': 5,
 'a': 6,
 'c': 7,
 'e': 8,
 'f': 9,
 'g': 10,
 'i': 11,
 'k': 12,
 'n': 13,
 'o': 14,
 'r': 15,
 's': 16,
 't': 17,
 'x': 18,
 'z': 19}

In [39]:
input_ids = [token2idx[token] for token in tokenized_text]
input_ids

[5,
 14,
 12,
 8,
 13,
 11,
 19,
 11,
 13,
 10,
 0,
 17,
 8,
 18,
 17,
 0,
 11,
 16,
 0,
 6,
 0,
 7,
 14,
 15,
 8,
 0,
 17,
 6,
 16,
 12,
 0,
 14,
 9,
 0,
 3,
 2,
 4,
 1]

In [40]:
# 上記のままではただのid列で、各トークンのかけ合わせをベクトルとして表現できない
# ∴one-hotベクトルに変換する必要がある。
"""
トークン: [c,b,a]
ただのid: [2,1,0]
one-hotベクトル: 
[
    [0, 0, 1], # 2 = c
    [0, 1, 0], # 1 = b
    [1, 0, 0]  # 0 = a
]
"""

import torch
import torch.nn.functional as F

input_ids = torch.tensor(input_ids)
one_hot_encodings = F.one_hot(input_ids, num_classes=len(token2idx)) # num_classesを設定しないとone-hotベクトルが語彙の大きさよりも短くなってしまう=不揃いなテンソルができるため、必ず設定する。
one_hot_encodings.shape

torch.Size([38, 20])

In [41]:
print(f"Token: {tokenized_text[0]}")
print(f"Tensor index: {input_ids[0]}")
print(f"One-hot: {one_hot_encodings[0]}")

Token: T
Tensor index: 5
One-hot: tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


## 単語トークン化
文字毎ではなく、単語に分割する。  
単語は活用形や句読点を含むものが別々のものとして扱われてしまい、膨大な数の語彙が存在することになり、その語彙の数だけone-hotベクトルの次元数が大きくなるということになる。  
そのため次に挙げるサブワードトークンにより、文字トークン化と単語トークン化の良いとこどりをするのが一般的。  

## サブワードトークン化
頻出単語は一意なものとして、稀多喃語はより小さな単位に分割して処理するので、入力の長さを管理可能なサイズに保ちつつ、スペルミス等も処理できるようにする。  
文字トークンや単語トークンと違い、トークン化時に統計的ルールとアルゴリズムを組み合わせて、事前学習用のコーパスからトークン化を学習する必要がある。  
BERTやDistilBERTのトークナイザーではWordPieceというトークン化アルゴリズムが用いられている。  

In [42]:
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [43]:
from transformers import DistilBertTokenizer
distilbert_tokenizer = DistilBertTokenizer.from_pretrained(model_ckpt)

In [44]:
text = 'Tokenizing text is a core task of NLP.'
encoded_text = tokenizer(text)
print(encoded_text)

{'input_ids': [101, 19204, 6026, 3793, 2003, 1037, 4563, 4708, 1997, 17953, 2361, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [45]:
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

['[CLS]', 'token', '##izing', 'text', 'is', 'a', 'core', 'task', 'of', 'nl', '##p', '.', '[SEP]']


In [46]:
tokenizer.convert_tokens_to_string(tokens)

'[CLS] tokenizing text is a core task of nlp. [SEP]'

In [47]:
# Tokenizerの語彙数
tokenizer.vocab_size

30522

In [48]:
# モデルの最大コンテキストサイズ
tokenizer.model_max_length

512

In [49]:
# モデルの入力層
tokenizer.model_input_names

['input_ids', 'attention_mask']

# データセット全体のトークン化
DatasetDictオブジェクトのmap()メソッドを使用すると便利

In [50]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)
# padding=True ... バッチ内で最大長のベクトルまでゼロで埋める。
# truncation=True ... トークンをモデルの最大コンテキストサイズまでとする。

In [52]:
from datasets import load_dataset
emotions = load_dataset("dair-ai/emotion")

In [54]:
tokenize(emotions["train"][:2])
# 0はpaddingで追加されたトークン
# attention-mask は、モデルが入力のパディング部分を無視できるように設定されている。∴attention-mask=0はパディング箇所

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 2064, 2175, 2013, 3110, 2061, 20625, 2000, 2061, 9636, 17772, 2074, 2013, 2108, 2105, 2619, 2040, 14977, 1998, 2003, 8300, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [60]:
# パディングしている"0"は、[PAD]トークンに該当する
tokenizer.convert_ids_to_tokens(tokenize(emotions["train"][:2])['input_ids'][0])

['[CLS]',
 'i',
 'didn',
 '##t',
 'feel',
 'humiliated',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [61]:
# コーパスすべてをトークナイズし、バッチに分割
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)
# batched=True ... 一括エンコード、デフォルトでは個別に動作することを適当に複数にまとめて処理するため、処理速度が上がるらしい。
# batch_size=None ... emotionsデータセット全体を1バッチとして適用
emotions_encoded["train"].column_names

Map: 100%|██████████| 2000/2000 [00:00<00:00, 35143.33 examples/s]


['text', 'label', 'input_ids', 'attention_mask']